# チュートリアル: WikipediaのHTMLページから歴代アメリカ大統領の[出生地,前任者,後任者,受賞歴]を抽出する
このパイプラインは、fonduer-tutorialsの[president_place_of_birth_tutorial.ipynb](https://github.com/HazyResearch/fonduer-tutorials/blob/master/wiki/president_place_of_birth_tutorial.ipynb)をベース(DeepL翻訳)に、抽出項目など修正を加えて作成したものです。

## postgresql準備
すでにpostgresqlにデータベースpob_presidentsが存在する場合は、1行目のコメントを外してドロップし、再作成してください。

In [1]:
! dropdb -h fonduer-postgres --if-exists pob_presidents #Uncomment to remove databases created in previous runs
! createdb -h fonduer-postgres pob_presidents

## Introduction

Webページ、ビジネスレポート、製品仕様書、科学文献に見られるような、テキスト、構造・表形式、視覚表現の組み合わせによって情報が伝達される[**リッチフォーマット**データ](https://hazyresearch.github.io/snorkel/blog/fonduer.html)から、`Fonduer`を使って関係を抽出する過程を説明します。

このチュートリアルでは、`Fonduer`を使って、大統領と副大統領の情報をwikipediaの記事から同定します。
HTMLファイルはWikipediaのマークアップ言語Wikitextから[wtf_wikipedia](https://github.com/spencermountain/wtf_wikipedia)を経由して生成されています。

チュートリアルはいくつかのパートに分かれており、それぞれが[Fonduer論文](https://arxiv.org/abs/1703.05028)で概説された`Fonduer`パイプラインのフェーズと、KBCプロセスをカバーしています。

1. KBC(Knowledge Base Construction)の初期化
2. メンション抽出とマルチモーダル特徴量によるcandidate抽出
3. 確率的関係性分類
4. テストセットでの評価

さらに、ユーザがラベリング関数を繰り返し改善することで、情報抽出の精度を向上させる方法を紹介します。

## Ph1：KBC(Knowledge Base Construction)の初期化

`Fonduer` のパイプラインの最初のフェーズでは、`Fonduer` はユーザが指定した`schema`を用いて、出力されるKBが格納されるリレーショナルデータベースを初期化します。
さらに、`Fonduer`は入力されたコーパスを繰り返し処理し、各文書を統一的なデータモデルに変換します。
このモデルはリッチフォーマットなデータの多様性とマルチモーダル性を捉えることができます。
この統一的なデータモデルは、残りのフェーズで使われる中間表現として機能します。

この前処理されたデータは、データベースに保存されます。
データベースへの接続文字列は `Meta`オブジェクトに渡され、並列実行のために`PostgreSQL`データベースを初期化します。

便宜上、データベースを呼び出すための変数と、`Fonduer`パイプラインをどのレベルの並列化で実行するかを定義するための変数をいくつか初期化します。

続行する前に、`PostgreSQL`がインストールされていることと、`pob_presidents`という名前の新しいデータベースが作成されていることを確認してください。

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "pob_presidents"
conn_string = 'postgresql://fonduer-postgres:5432/' + ATTRIBUTE

### 1.1 入力文書の解析と統一データモデルへの変換
このオブジェクトはデータベースとの接続を自動的に管理し、中間結果を保存することができます。

In [3]:
from fonduer import Meta, init_logging

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

[2022-04-17 03:27:28,553][INFO] fonduer.meta:49 - Setting logging directory to: logs/2022-04-17_03-27-28
[2022-04-17 03:27:28,608][INFO] fonduer.meta:135 - Connecting user:None to fonduer-postgres:5432/pob_presidents
[2022-04-17 03:27:29,907][INFO] fonduer.meta:162 - Initializing the storage schema


次に、wikpediaの記事コーパスを読み込み、統一的なデータモデルに変換します。
各記事はHTMLファイルで表現されます。
このHTMLファイルを解析し、テキスト、構造、表形式のモダリティ情報を持つロバストな統一データモデルを作成します。
各文書は互いに独立しているので、並行して解析することができることに注意してください。
この機能については`PostgreSQL`に依存しています。

#### HTMLDocPreprocessorの設定
まず、ドキュメントが格納されるパスを設定し、指定したパスにあるドキュメントを読み込むための `HTMLDocPreprocessor` を定義します。
`max_docs` にはパースするドキュメントの最大数を指定します。

**注意: 次のステップを実行する前に`download_data.sh`を実行しておく必要があります。(docker-compose実行時に実行済み)

In [4]:
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser

docs_path = "../../tutorial_president_wiki/data/presidents/"
doc_preprocessor = HTMLDocPreprocessor(docs_path)

#### パーサーの設定
次に、コーパスパーサーである`Parser`を設定します。
前処理として[spaCy](https://spacy.io/)を用いて、文書を文とトークンに分割します。
また、各文書の統一データモデルに含めるモダリティ情報を指定することが可能です。
以下では、構造情報だけでなく、[spaCy]を用いてこれらの文の品詞タグや依存構文構造などのアノテーションを提供する言語情報も有効にしています。
なお、構文解析の完了を示す進捗バーが表示された後、すべてのオブジェクトがデータベースに挿入されるまで、さらに時間がかかることに注意してください。

In [5]:
corpus_parser = Parser(session, structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

[2022-04-17 03:27:30,906][INFO] fonduer.utils.udf:67 - Running UDF...


  0%|          | 0/78 [00:00<?, ?it/s]

CPU times: user 18.2 s, sys: 1.36 s, total: 19.6 s
Wall time: 6min 46s


そして、簡単なデータベースクエリ（`Fonduer`が使っている[SQLAlchemy](http://www.sqlalchemy.org/)の構文で書かれています）を使って、いくつの文書や文が解析されたか、あるいはそれぞれの文書にいくつの文が含まれているかをチェックすることができます。

In [6]:
from fonduer.parser.models import Document, Sentence

print(f"Documents: {session.query(Document).count()}")
print(f"Sentences: {session.query(Sentence).count()}")

Documents: 78
Sentences: 98397


### 1.2 コーパスをテストとトレーニングに分割する
ドキュメントを70:15:15の割合でtrain:dev:testに分割することにします。
ここでは、チュートリアルデータの一貫性を逆手に取り、非ランダムな順序で分割を行います。

In [7]:
docs = session.query(Document).order_by(Document.name).all()
ld = len(docs)

train_docs = set()
dev_docs = set()
test_docs = set()
splits = (0.7, 0.85)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)

from pprint import pprint

print("train:")
pprint([x.name for x in train_docs])
print("dev:")
pprint([x.name for x in dev_docs])
print("test:")
pprint([x.name for x in test_docs])

train:
['James_Buchanan',
 'Charles_Curtis',
 'Chester_A._Arthur',
 'George_H._W._Bush',
 'Hubert_Humphrey',
 'Dwight_D._Eisenhower',
 'John_C._Calhoun',
 'Levi_P._Morton',
 'Dick_Cheney',
 'George_Washington',
 'John_Tyler',
 'Adlai_Stevenson',
 'Barack_Obama',
 'Henry_Wilson',
 'Alben_W._Barkley',
 'Gerald_Ford',
 'Henry_A._Wallace',
 'Al_Gore',
 'Dan_Quayle',
 'Donald_Trump',
 'James_Madison',
 'Benjamin_Harrison',
 'Harry_S._Truman',
 'John_F._Kennedy',
 'John_Quincy_Adams',
 'Charles_G._Dawes',
 'James_A._Garfield',
 'Abraham_Lincoln',
 'Elbridge_Gerry',
 'John_Adams',
 'George_W._Bush',
 'John_C._Breckinridge',
 'Franklin_Pierce',
 'James_K._Polk',
 'George_Clinton',
 'Aaron_Burr',
 'George_M._Dallas',
 'Grover_Cleveland',
 'John_Nance_Garner',
 'Calvin_Coolidge',
 'Daniel_D._Tompkins',
 'Hannibal_Hamlin',
 'Joe_Biden',
 'Andrew_Johnson',
 'Charles_W._Fairbanks',
 'Jimmy_Carter',
 'Franklin_D._Roosevelt',
 'James_Monroe',
 'James_S._Sherman',
 'Garret_Hobart',
 'Lyndon_B._Johnson

## Ph2：メンション抽出とマルチモーダル特徴量によるcandidate抽出

フェーズ1より統一的なデータモデルが与えられると、`Fonduer`はユーザーから提供された`matchers`と`throttlers`に基づく`candidate`を抽出します。
次に`Fonduer`は統一されたデータから得られるマルチモダリティな情報を活用して、各候補のマルチモーダルな特徴を提供します。

### 2.1 メンションの抽出

まず、コーパスからメンションを抽出します。
メンションとは、`candidate`を構成するオブジェクトの種類です。
例えば、(副)大統領の名前とそれに対応する出生地のペアを抽出したい場合、名前が1つのメンションとなり、出生地がもう1つのメンションとなります。
これらのメンションを組み合わせて`candidate`を作成し、関連する文書においてどの`candidate`が正しいかを予測します。

まず、抽出したいメンションの定義と命名を始めます。
今回は「名前、出生地、前任者、後任者、受賞歴」を抽出していきます。

In [8]:
from fonduer.candidates.models import mention_subclass

Presidentname = mention_subclass("Presidentname")
Placeofbirth = mention_subclass("Placeofbirth")
Predecessor = mention_subclass("Predecessor")
Successor = mention_subclass("Successor")
Awards = mention_subclass("Awards")

次に、コーパス中のどのテキストスパンが各エンティティのインスタンスであるかを定義するために、`matcher`を記述します。
`matcher`は正規表現、辞書、ユーザー定義関数など様々な情報を利用することができます。
異なる技術を組み合わせることで、より質の高い`matcher`を作成することができます。
一般に、`matcher`は完全な再現性を維持しながら、可能な限り正確であることを追求する必要があります。
`matcher`に関する詳しいドキュメントは、[Read the Docs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#matchers)に掲載されています。

抽出対象に対応する文字列を定義する`matcher`を記述する必要があります。

#### 大統領名のmatcher作成

大統領名の`matcher`は、htmlファイル名に大統領の名前が表示されていることを利用します。
大統領についてのメンションを見つけるために、「テキストの`span`をファイル名と比較する関数」を定義します。
すでに文書を統一的なデータモデルに変換したので、テキストの各`span`はソースの文書名を参照することができ、ソースファイル名との比較が容易です。

In [9]:
def mention_span_matches_file_name(mention):
    president_name_string = mention.get_span()
    file_name = mention.sentence.document.name.replace("_", " ")
    if president_name_string == file_name:
        return True
    else:
        return False

この関数を `LambdaFunctionMatcher` を使って、 `Fonduer` に言及する`matcher`にラップします。

In [10]:
from fonduer.candidates.matchers import LambdaFunctionMatcher, Intersect, Union

president_name_matcher = LambdaFunctionMatcher(
    func=mention_span_matches_file_name
)

#### 出生地のmatcher作成

出生地は正規性表現による指定も可能ですが、ここでは文書内の位置に基づいてマッチングを行います。
htmlファイルを確認すると、各ファイルにはそれぞれの大統領に関する最も重要な情報が記載された表が記載されています。
表の左の列にはラベル、右の列にはそれに対応する値が存在しています。
この表情報を取得するために、「`span`が表中にあり、同じ行の隣接するセルに「出生地」のラベルがあるかどうか」をチェックする関数を定義します。

htmlファイルを確認すると、出生地は、市町村,州,国 の状態に分割されていることがわかります。
(例：Birth place : Honolulu, Hawaii, U.S.)

ここで、市町村と州の２つの情報を抽出していきます。

In [11]:
from fonduer.utils.data_model_utils import get_row_ngrams


def is_in_birthplace_table_row(mention):
    if not mention.sentence.is_tabular():
        return False
    ngrams = get_row_ngrams(mention, lower=True)
    birth_place_words = set(["birth", "place"])
    if birth_place_words <= set(ngrams):
        return True
    else:
        return False


def birthplace_left_aligned_to_punctuation(mention):
    # Return false, if the cell containing the text candidate does not have any reference
    # to `sentence` objects
    for sentence in mention.sentence.cell.sentences:
        sentence_parts = sentence.text.split(",")
        for sentence_part in sentence_parts:
            if sentence_part.startswith(mention.get_span()):
                return True
    return False


# We only want one granularity of the birth place
def no_commas_in_birth_place(mention):
    if "," in mention.get_span():
        return False
    else:
        return True

出生地のプロパティを取得するためのすべての関数を定義した後、`Intersect matcher`を介してそれらを結合します。
この`matcher`は、3つの関数がすべて一致した場合にのみ`span`を選択します。

In [12]:
birth_place_in_labeled_row_matcher = LambdaFunctionMatcher(
    func=is_in_birthplace_table_row
)

birth_place_in_labeled_row_matcher.longest_match_only = False

birth_place_no_commas_matcher = LambdaFunctionMatcher(
    func=no_commas_in_birth_place
)

birth_place_left_aligned_matcher = LambdaFunctionMatcher(
    func=birthplace_left_aligned_to_punctuation
)

place_of_birth_matcher = Intersect(
    birth_place_in_labeled_row_matcher,
    birth_place_no_commas_matcher,
    birth_place_left_aligned_matcher
)

#### 前任者のmatcher作成

大統領職の前任者の名前を、htmlファイルの表から取得していきます。
(例：Predecessor : George W. Bush)

大統領によっては、Predecessor, Predecessor2, Predecessor3,,と、複数名の記載がある場合があります。

In [13]:
def is_in_predecessor_table_row(mention):
    if not mention.sentence.is_tabular():
        return False
    ngrams = get_row_ngrams(mention, lower=True)
    predecessor_words = set(["Predecessor"])
    if predecessor_words <= set(ngrams):
        return True
    else:
        return False


def predecessor_left_aligned_to_punctuation(mention):
    for sentence in mention.sentence.cell.sentences:
        sentence_parts = sentence.text.split(",")
        for sentence_part in sentence_parts:
            if sentence_part.startswith(mention.get_span()):
                return True
    return False

In [14]:
predecessor_in_labeled_row_matcher = LambdaFunctionMatcher(
    func=is_in_predecessor_table_row
)

predecessor_in_labeled_row_matcher.longest_match_only = False

predecessor_left_aligned_matcher = LambdaFunctionMatcher(
    func=predecessor_left_aligned_to_punctuation
)

predecessor_matcher = Intersect(
    predecessor_in_labeled_row_matcher,
)

#### 後任者のmatcher作成

大統領職の後任者の名前を、htmlファイルの表から取得していきます。
(例：Successor : Donald Trump)

大統領によっては、Successor, Successor2, Successor3,,と、複数名の記載がある場合があります。

In [15]:
def is_in_successor_table_row(mention):
    if not mention.sentence.is_tabular():
        return False
    ngrams = get_row_ngrams(mention, lower=True)
    successor_words = set(["Successor"])
    if successor_words <= set(ngrams):
        return True
    else:
        return False


def successor_left_aligned_to_punctuation(mention):
    for sentence in mention.sentence.cell.sentences:
        sentence_parts = sentence.text.split(",")
        for sentence_part in sentence_parts:
            if sentence_part.startswith(mention.get_span()):
                return True
    return False

In [16]:
successor_in_labeled_row_matcher = LambdaFunctionMatcher(
    func=is_in_successor_table_row
)

successor_in_labeled_row_matcher.longest_match_only = False

successor_left_aligned_matcher = LambdaFunctionMatcher(
    func=successor_left_aligned_to_punctuation
)

successor_matcher = Intersect(
    successor_in_labeled_row_matcher,
)

#### 受賞歴のmatcher作成

大統領の受賞歴を、htmlファイルの表から取得していきます。
(例：Awards : Nobel Peace Prize (2009) Profile in Courage Award (2017))

Awardはすべての大統領に記載があるわけではありません。
また、Awardに受賞年は必ずとも記載されていません。

In [17]:
def is_in_awards_table_row(mention):
    if not mention.sentence.is_tabular():
        return False
    ngrams = get_row_ngrams(mention, lower=True)
    awards_words = set(["Awards"])
    if successor_words <= set(ngrams):
        return True
    else:
        return False


def awards_left_aligned_to_punctuation(mention):
    for sentence in mention.sentence.cell.sentences:
        sentence_parts = sentence.text.split(",")
        for sentence_part in sentence_parts:
            if sentence_part.startswith(mention.get_span()):
                return True
    return False

In [18]:
awards_in_labeled_row_matcher = LambdaFunctionMatcher(
    func=is_in_awards_table_row
)

awards_in_labeled_row_matcher.longest_match_only = False

awards_left_aligned_matcher = LambdaFunctionMatcher(
    func=awards_left_aligned_to_punctuation
)

awards_matcher = Intersect(
    awards_in_labeled_row_matcher,
)

#### メンション空間(MentionSpace)の定義

次に、ドキュメントから考えられるすべてのメンションの「空間」を定義するために、抽出したい関係の各要素に対して`MentionSpace`を定義していきます。
`Fonduer`はデフォルトの`MentionSpace`を用意していますが、必要に応じてデフォルトの`MentionSpace`を拡張することも可能です。

名前の場合、`MentionSpace`は比較的単純で、それぞれの名前には少なくとも2つの単語（名、姓）が含まれています。
ミドルネームの追加を考慮すると、1つの名前に最大で4つの単語が含まれることが予想されます。

同様に、出生地は1語から3語の`span`であることが予想されます。

前任者と後任者については、大統領の名前なので、最大で4つの単語が含まれることが予想されます。
また、キャリアによって、複数名の前任者と後任者を持つ可能性があります。

受賞歴については、2語以上であることが予想されます。

これらのプロパティを定義するために、`fonduer`が提供するデフォルトの`Ngrams`クラスを使用します。

In [19]:
from fonduer.candidates import MentionNgrams

presname_ngrams = MentionNgrams(n_max=4, n_min=2)
placeofbirth_ngrams = MentionNgrams(n_max=3)
predecessor_ngrams = MentionNgrams(n_max=4, n_min=2)
successor_ngrams = MentionNgrams(n_max=4, n_min=2)
awards_ngrams = MentionNgrams(n_min=2)

#### メンション抽出の実行 

次に、上記で定義した`MentionSpace`と`matcher`を元に、全ての文書からメンションを抽出する`MentionExtractor`を作成します。

`MentionExtractor`のAPIは[ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#fonduer.candidates.MentionExtractor) で確認することができます。

In [22]:
from fonduer.candidates import MentionExtractor

"""
mention_extractor = MentionExtractor(
    session,
    [Presidentname, Placeofbirth, Predecessor, Successor, Awards],
    [presname_ngrams, placeofbirth_ngrams, predecessor_ngrams, successor_ngrams, awards_ngrams],
    [president_name_matcher, place_of_birth_matcher, predecessor_matcher, successor_matcher, awards_matcher],
)
"""

mention_extractor = MentionExtractor(
    session,
    [Presidentname, Placeofbirth, Awards],
    [presname_ngrams, placeofbirth_ngrams, Awards],
    [president_name_matcher, place_of_birth_matcher, Awards]
)


すべての文書に対して`extractor`を実行します。

In [ ]:
from fonduer.candidates.models import Mention

mention_extractor.apply(docs, parallelism=PARALLEL)
num_names = session.query(Presidentname).count()
num_pobs = session.query(Placeofbirth).count()
num_prede = session.query(Predecessor).count()
num_succe = session.query(Successor).count()
num_award = session.query(Awards).count()

print(
    f"Total Mentions: {session.query(Mention).count()} ({num_names} names, {num_pobs} places of birth, {num_prede} Predecessor, {num_succe} Successor, {num_award} Awards)"
)

[2022-04-17 04:27:05,232][INFO] fonduer.candidates.mentions:467 - Clearing table: presidentname
[2022-04-17 04:27:05,253][INFO] fonduer.candidates.mentions:467 - Clearing table: placeofbirth
[2022-04-17 04:27:05,265][INFO] fonduer.candidates.mentions:467 - Clearing table: awards
[2022-04-17 04:27:05,269][INFO] fonduer.utils.udf:67 - Running UDF...


  0%|          | 0/78 [00:00<?, ?it/s]

Process MentionExtractorUDF-15:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/user/.venv/lib/python3.7/site-packages/fonduer/utils/udf.py", line 222, in run
    y = self.apply(doc, **self.apply_kwargs)
  File "/home/user/.venv/lib/python3.7/site-packages/fonduer/candidates/mentions.py", line 567, in apply
    for child_context in self.matchers[i].apply(
AttributeError: type object 'Awards' has no attribute 'apply'
Process MentionExtractorUDF-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/user/.venv/lib/python3.7/site-packages/fonduer/utils/udf.py", line 222, in run
    y = self.apply(doc, **self.apply_kwargs)
  File "/home/user/.venv/lib/python3.7/site-packages/fonduer/candidates/mentions.py", line 567, in apply
    for child_context in self.matchers[i].apply(
AttributeError: ty

### 2.2 candidateの抽出

`candidate`を構成するためのメンションの定義と抽出ができたので、次は`candidate`の抽出を行います。
メンションと同様に、まず各候補のスキーマを定義します。
この例では、上記で定義した`Presidentname`と、`Placeofbirth`,`Predecessor`,`Successor`,`Awards`のメンションからなる候補を作成します。
この`candidate`をそれぞれ`Presidentname_Placeofbirth`,`Presidentname_Predecessor`,`Presidentname_Successor`,`Presidentname_Awards`と名付けます。

In [ ]:
from fonduer.candidates.models import candidate_subclass

Presidentname_Placeofbirth = candidate_subclass(
    "Presidentname_Placeofbirth", [Presidentname, Placeofbirth]
)

Presidentname_Predecessor = candidate_subclass(
    "Presidentname_Predecessor", [Presidentname, Predecessor]
)

Presidentname_Successor = candidate_subclass(
    "Presidentname_Successor", [Presidentname, Successor]
)

Presidentname_Awards = candidate_subclass(
    "Presidentname_Awards", [Presidentname, Awards]
)

#### Throttlersの定義

次に、余分な`candidate`を減らし、無効な`candidate`を不必要に対象にしないためのThrottlersを定義します。
`Throttler`は、`matcher`のようにフィルターとして機能し、取りこぼしはしない程度のフィルタとして作成するべきです。

ここでは、同じ表中にある`candidate`を破棄する`Throttler`を作成します。
(ただし、現在この`Throttler`は使っていません。`wtf_wikipedia`の問題で、大統領名がテーブルに存在しないことがあるからです。)

そのため、`Throttler`はスキップしています。
他の例でも`Throttler`をスキップすることは多いです。

#### CandidateExtractorの実行

これで、`candidate`抽出に必要なコンポーネントはすべて揃いました。
各`candidate`を構成する`mentions`と、余分な候補を取り除くための`throttler`を定義しました。
ここでは`CandidateExtractor`を定義して、使用する`candidate`のサブクラスと対応する`throttler`を定義します。

`CandidateExtractor`のAPIは [ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#fonduer.candidates.CandidateExtractor)で確認することができます。

In [ ]:
from fonduer.candidates import CandidateExtractor

candidate_extractor_Presidentname_Placeofbirth = CandidateExtractor(session, [Presidentname_Placeofbirth])
candidate_extractor_Presidentname_Predecessor = CandidateExtractor(session, [Presidentname_Predecessor])
candidate_extractor_Presidentname_Successor = CandidateExtractor(session, [Presidentname_Successor])
candidate_extractor_Presidentname_Awards = CandidateExtractor(session, [Presidentname_Awards])

ここでは、`split=0`を指定して、これらの`candidates`がトレーニングセットに属することを指定しています。
ここで、train:dev:testを[0]:[1]:[2]で分割して参照していることを思い出してください。

まずは`Presidentname_Placeofbirth`で実施します。

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
    candidate_extractor_Presidentname_Placeofbirth.apply(docs, split=i, parallelism=PARALLEL)
    print(
        f"Number of Candidates in split={i}: {session.query(Presidentname_Placeofbirth).filter(Presidentname_Placeofbirth.split == i).count()}"
    )

train_cands_Presidentname_Placeofbirth = candidate_extractor_Presidentname_Placeofbirth.get_candidates(split=0)
dev_cands_Presidentname_Placeofbirth = candidate_extractor_Presidentname_Placeofbirth.get_candidates(split=1)
test_cands_Presidentname_Placeofbirth = candidate_extractor_Presidentname_Placeofbirth.get_candidates(split=2)

次に`Presidentname_Predecessor`で実施します。

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
    candidate_extractor_Presidentname_Predecessor.apply(docs, split=i, parallelism=PARALLEL)
    print(
        f"Number of Candidates in split={i}: {session.query(Presidentname_Predecessor).filter(Presidentname_Predecessor.split == i).count()}"
    )

train_cands_Presidentname_Predecessor = candidate_extractor_Presidentname_Predecessor.get_candidates(split=0)
dev_cands_Presidentname_Predecessor = candidate_extractor_Presidentname_Predecessor.get_candidates(split=1)
test_cands_Presidentname_Predecessor = candidate_extractor_Presidentname_Predecessor.get_candidates(split=2)

次に`Presidentname_Successor`で実施します。

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
    candidate_extractor_Presidentname_Successor.apply(docs, split=i, parallelism=PARALLEL)
    print(
        f"Number of Candidates in split={i}: {session.query(Presidentname_Successor).filter(Presidentname_Successor.split == i).count()}"
    )

train_cands_Presidentname_Successor = candidate_extractor_Presidentname_Successor.get_candidates(split=0)
dev_cands_Presidentname_Successor = candidate_extractor_Presidentname_Successor.get_candidates(split=1)
test_cands_Presidentname_Successor = candidate_extractor_Presidentname_Successor.get_candidates(split=2)

最後に`Presidentname_Awards`で実施します。

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
    candidate_extractor_Presidentname_Awards.apply(docs, split=i, parallelism=PARALLEL)
    print(
        f"Number of Candidates in split={i}: {session.query(Presidentname_Awards).filter(Presidentname_Awards.split == i).count()}"
    )

train_cands_Presidentname_Awards = candidate_extractor_Presidentname_Awards.get_candidates(split=0)
dev_cands_Presidentname_Awards = candidate_extractor_Presidentname_Awards.get_candidates(split=1)
test_cands_Presidentname_Awards = candidate_extractor_Presidentname_Awards.get_candidates(split=2)

### 2.2 マルチモーダルな特徴づけ

構造化されていないテキストを扱うのとは異なり、`Fonduer`はリッチにフォーマットされたデータを扱うため、各`candidate`をマルチモーダル特徴の基本ライブラリで特徴量化します。

#### Postgres Featurizerによる特徴量化

次に、train:dev:test(学習セット、開発セット、テストセット)の`candidate`に特徴量のアノテーションを施します。
`Fonduer` が提供する`Featurizer`を使うと、この作業を並行して行うことができ、パフォーマンスを向上させることができます。

`Featurizer` が提供する API は [ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/features.html#fonduer.features.Featurizer) で確認することができます。

まずは`Presidentname_Placeofbirth`で実施します。

In [ ]:
from fonduer.features import Featurizer

featurizer_Presidentname_Placeofbirth = Featurizer(session, [Presidentname_Placeofbirth])
%time featurizer_Presidentname_Placeofbirth.apply(split=0, train=True, parallelism=PARALLEL)
%time F_train_Presidentname_Placeofbirth = featurizer_Presidentname_Placeofbirth.get_feature_matrices(train_cands_Presidentname_Placeofbirth)

In [ ]:
print(F_train_Presidentname_Placeofbirth[0].shape)
%time featurizer_Presidentname_Placeofbirth.apply(split=1, parallelism=PARALLEL)
%time F_dev_Presidentname_Placeofbirth = featurizer_Presidentname_Placeofbirth.get_feature_matrices(dev_cands_Presidentname_Placeofbirth)
print(F_dev_Presidentname_Placeofbirth[0].shape)

In [ ]:
%time featurizer_Presidentname_Placeofbirth.apply(split=2, parallelism=PARALLEL)
%time F_test_Presidentname_Placeofbirth = featurizer_Presidentname_Placeofbirth.get_feature_matrices(test_cands_Presidentname_Placeofbirth)
print(F_test_Presidentname_Placeofbirth[0].shape)

次に`Presidentname_Predecessor`で実施します。

In [ ]:
featurizer_Presidentname_Predecessor = Featurizer(session, [Presidentname_Predecessor])
%time featurizer_Presidentname_Predecessor.apply(split=0, train=True, parallelism=PARALLEL)
%time F_train_Presidentname_Predecessor = featurizer_Presidentname_Predecessor.get_feature_matrices(train_cands_Presidentname_Predecessor)

In [ ]:
print(F_train_Presidentname_Predecessor[0].shape)
%time featurizer_Presidentname_Predecessor.apply(split=1, parallelism=PARALLEL)
%time F_dev_Presidentname_Predecessor = featurizer_Presidentname_Predecessor.get_feature_matrices(dev_cands_Presidentname_Predecessor)
print(F_dev_Presidentname_Predecessor[0].shape)

In [ ]:
%time featurizer_Presidentname_Predecessor.apply(split=2, parallelism=PARALLEL)
%time F_test_Presidentname_Predecessor = featurizer_Presidentname_Predecessor.get_feature_matrices(test_cands_Presidentname_Predecessor)
print(F_test_Presidentname_Predecessor[0].shape)

次に`Presidentname_Successor`で実施します。

In [ ]:
featurizer_Presidentname_Successor = Featurizer(session, [Presidentname_Successor])
%time featurizer_Presidentname_Successor.apply(split=0, train=True, parallelism=PARALLEL)
%time F_train_Presidentname_Successor = featurizer_Presidentname_Successor.get_feature_matrices(train_cands_Presidentname_Successor)

In [ ]:
print(F_train_Presidentname_Successor[0].shape)
%time featurizer_Presidentname_Successor.apply(split=1, parallelism=PARALLEL)
%time F_dev_Presidentname_Successor = featurizer_Presidentname_Successor.get_feature_matrices(dev_cands_Presidentname_Successor)
print(F_dev_Presidentname_Successor[0].shape)

In [ ]:
%time featurizer_Presidentname_Successor.apply(split=2, parallelism=PARALLEL)
%time F_test_Presidentname_Successor = featurizer_Presidentname_Successor.get_feature_matrices(test_cands_Presidentname_Successor)
print(F_test_Presidentname_Successor[0].shape)

最後に`Presidentname_Awards`で実施します。

In [ ]:
featurizer_Presidentname_Awards = Featurizer(session, [Presidentname_Awards])
%time featurizer_Presidentname_Awards.apply(split=0, train=True, parallelism=PARALLEL)
%time F_train_Presidentname_Awards = featurizer_Presidentname_Awards.get_feature_matrices(train_cands_Presidentname_Awards)

In [ ]:
print(F_train_Presidentname_Awards[0].shape)
%time featurizer_Presidentname_Awards.apply(split=1, parallelism=PARALLEL)
%time F_dev_Presidentname_Awards = featurizer_Presidentname_Awards.get_feature_matrices(dev_cands_Presidentname_Awards)
print(F_dev_Presidentname_Awards[0].shape)

In [ ]:
%time featurizer_Presidentname_Awards.apply(split=2, parallelism=PARALLEL)
%time F_test_Presidentname_Awards = featurizer_Presidentname_Awards.get_feature_matrices(test_cands_Presidentname_Awards)
print(F_test_Presidentname_Awards[0].shape)

このフェーズでは、`candidate`の集合と特徴行列を生成します。
なお、フェーズ1と2は比較的静的な処理であり、通常、KBCの処理中に一度だけ実行されるものです。

## Ph3: 確率的関係性分類
このフェーズでは、ユーザが定義した「ラベリング関数」を適用します。
このラベリング関数には、データにラベルを付けるための様々なヒューリスティックなパターン、および [weak supervision](http://hazyresearch.github.io/snorkel/blog/weak_supervision.html)が含まれており、各`candidate`に対して、データプログラミングエンジンが使用するラベル行列を作成します。

一般に、手作業でラベル付けされた学習データはまれであり、高価になります。
ここでは、大量のラベルなし学習データを対象に、それらをラベリング関数記述でラベル付けしていきます。
ラベル付けする知識の例として、「「New」「County」「City」といった単語が含まれると都市名である」といったような知識をラベリング関数として記述します。
ラベリング関数は完全ではなく、中には非常に低品質なものが記述される場合もあるため、生成モデルを用いてその精度をモデル化していくことになります。

そして、データプログラミングを用いて機械学習モデルを学習し、`candidate`を分類する際にどの特徴が最も重要であるかを学習することができます。

### ゴールドデータ(正解データ)の読み込み
エラー解析や評価を実施するため、このチュートリアルで使用するdevセットとtestセットには既にアノテーションを施してあります。
外部で定義されたヘルパー関数を使って読み込むことにします。

現在、このゴールドデータは、`Presidentname_Placeofbirth`しか実装していません。

In [ ]:
from fonduer.supervision.models import GoldLabel
from wiki_table_utils import gold

from fonduer.supervision import Labeler

docs = corpus_parser.get_documents()
labeler_pob_city = Labeler(session, [Presidentname_Placeofbirth])
%time labeler_pob_city.apply(docs=docs, lfs=[[gold]], table=GoldLabel, train=True, parallelism=PARALLEL)

### ラベリング関数の作成

`Fonduer`では、学習する抽出モデルに学習シグナルを提供するための主要なインターフェースは、ラベリング関数(LF)を書くことです。
これは、大量の学習セットを手でラベリングするのとは対照的な作業です。

ラベリング関数は特別なものではありません。
`candidate`を入力引数として受け取り、`candidate`を「真/TRUE」とすべきと判断した場合は「1」、「偽/FALSE」とすべきと判断した場合は「0」、「棄権/ABSTAIN」した場合は「-1」を返すPythonの関数に過ぎないのです。
実際には、多くのLFは単極性であり、TRUEかABSTAINを返すLF、FALSEかABSTAINを返すLFがほとんどです。
TRUEかFALSEを返すLF、もしくはTRUEかFALSEかABSTAINを返すLFは、別のLFとして分けた方がよいです。

`Fonduer`を使えば、ラベリング関数を対象コーパスを調査して発見した直感的なパターンを使って書くことができます。
ラベリング関数を書くために使えるデータモデルユーティリティのライブラリは [Read the Docs](http://fonduer.readthedocs.io/en/stable/user/data_model_utils.html) に概要がまとめられています。

以下、このような単純なパターンのいくつかをラベリング関数の集合として表現していきます。

まずは、`Presidentname_Placeofbirth`のLFです。

In [ ]:
from snorkel.labeling import labeling_function
from fonduer.utils.data_model_utils import *
import re

ABSTAIN = -1
FALSE = 0
TRUE = 1


@labeling_function()
def LF_place_of_birth_has_link(c):
    place = c.placeofbirth
    ancestor_tag_names = get_ancestor_tag_names(place)
    if len(ancestor_tag_names) > 1 and "a" in ancestor_tag_names:
        return ABSTAIN
    else:
        return FALSE


@labeling_function()
def LF_place_of_birth_is_longest_ordered_span_before_comma(c):
    place = c.placeofbirth
    place_string = place.context.get_span()
    place_sentence_string = place.context.sentence.text
    left_aligned_first_span = place_sentence_string.split(",")[0]
    if place_string == left_aligned_first_span:
        return TRUE
    else:
        return FALSE


@labeling_function()
def LF_place_in_first_sentence_of_cell(c):
    place = c.placeofbirth
    place_sentence = place.context.sentence
    place_cell = place_sentence.cell
    if place_sentence == place_cell.sentences[0]:
        return TRUE
    else:
        return FALSE

    
@labeling_function()
def LF_place_in_second_sentence_of_cell(c):
    place = c.placeofbirth
    place_sentence = place.context.sentence
    place_cell = place_sentence.cell
    if place_sentence == place_cell.sentences[1]:
        return TRUE
    else:
        return FALSE


@labeling_function()
def LF_place_is_full_sentence(c):
    place = c.placeofbirth
    place_sentence = place.context.sentence
    if place.context.get_span() == place_sentence.text:
        return ABSTAIN
    else:
        return FALSE


@labeling_function()
def LF_place_not_a_US_state(c):
    place = c.placeofbirth
    place_string = place.context.get_span().lower()
    if place_string is None:
        return FALSE
    state_dictionary = set(
        x.lower()
        for x in [
            "Alabama",
            "Alaska",
            "Arizona",
            "Arkansas",
            "California",
            "Colorado",
            "Connecticut",
            "Delaware",
            "Florida",
            "Georgia",
            "Hawaii",
            "Idaho",
            "Illinois",
            "Indiana",
            "Iowa",
            "Kansas",
            "Kentucky",
            "Louisiana",
            "Maine",
            "Maryland",
            "Massachusetts",
            "Michigan",
            "Minnesota",
            "Mississippi",
            "Missouri",
            "Montana",
            "Nebraska",
            "Nevada",
            "New Hampshire",
            "New Jersey",
            "New Mexico",
            "New York",
            "North Carolina",
            "North Dakota",
            "Ohio",
            "Oklahoma",
            "Oregon",
            "Pennsylvania",
            "Rhode Island",
            "South Carolina",
            "South Dakota",
            "Tennessee",
            "Texas",
            "Utah",
            "Vermont",
            "Virginia",
            "Washington",
            "West Virginia",
            "Wisconsin",
            "Wyoming",
        ]
    )
    if place_string == "new york city":  # exception
        return TRUE
    if place_string in state_dictionary:
        return FALSE
    elif any(x in place_string for x in state_dictionary):
        return FALSE
    else:
        return ABSTAIN

次に、使いたいラベリング関数を全て一つのリストに集め、それを `Labeler` の入力として提供していきます。
市町村を取得するLFセットと、州を取得するLFセットを用意します。

In [ ]:
president_name_pob_city_lfs = [
    LF_place_of_birth_has_link,
    LF_place_of_birth_is_longest_ordered_span_before_comma,
    LF_place_not_a_US_state,
    LF_place_in_first_sentence_of_cell,
    LF_place_is_full_sentence,
]

In [ ]:
president_name_pob_state_lfs = [
    LF_place_of_birth_has_link,
    LF_place_of_birth_is_longest_ordered_span_before_comma,
    LF_place_in_second_sentence_of_cell,
    LF_place_is_full_sentence,
]

次に、`Presidentname_Predecessor`のLFになります。
`Fonduer`はLFセットは最低限３つ必要なので、ここではあえてダミーLFを用意してみます。

In [ ]:
@labeling_function()
def LF_predecessor_has_link(c):
    cand = c.predecessor
    ancestor_tag_names = get_ancestor_tag_names(cand)
    if len(ancestor_tag_names) > 1 and "a" in ancestor_tag_names:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_predecessor_is_full_sentence(c):
    cand = c.predecessor
    cand_sentence = cand.context.sentence
    if cand.context.get_span() == cand_sentence.text:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_predecessor_sentence_num(c):
    cand = c.predecessor
    cand_sentence = cand.context.sentence
    cand_cell = cand_sentence.cell
    if len(cand_sentence) >= 2:
        return TRUE
    else:
        return FALSE
    
@labeling_function()
def LF_predecessor_dummy(c):
    return TRUE

In [ ]:
president_name_predecessor_lfs = [
    LF_predecessor_has_link,
    LF_predecessor_is_full_sentence,
    LF_predecessor_sentence_num,
    LF_predecessor_dummy
]

次に、`Presidentname_Successor`のLFになります。

In [ ]:
@labeling_function()
def LF_successor_has_link(c):
    cand = c.successor
    ancestor_tag_names = get_ancestor_tag_names(cand)
    if len(ancestor_tag_names) > 1 and "a" in ancestor_tag_names:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_successor_is_full_sentence(c):
    cand = c.successor
    cand_sentence = cand.context.sentence
    if cand.context.get_span() == cand_sentence.text:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_successor_sentence_num(c):
    cand = c.successor
    cand_sentence = cand.context.sentence
    cand_cell = cand_sentence.cell
    if len(cand_sentence) >= 2:
        return TRUE
    else:
        return FALSE

@labeling_function()
def LF_successor_dummy(c):
    return TRUE

In [ ]:
president_name_successor_lfs = [
    LF_successor_has_link,
    LF_successor_is_full_sentence,
    LF_successor_sentence_num,
    LF_successor_dummy
]

最後に、`Presidentname_Awards`のLFになります。

In [ ]:
@labeling_function()
def LF_awards_has_link(c):
    cand = c.awards
    ancestor_tag_names = get_ancestor_tag_names(cand)
    if len(ancestor_tag_names) > 1 and "a" in ancestor_tag_names:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_awards_is_full_sentence(c):
    cand = c.awards
    cand_sentence = cand.context.sentence
    if cand.context.get_span() == cand_sentence.text:
        return ABSTAIN
    else:
        return FALSE

@labeling_function()
def LF_awards_sentence_num(c):
    cand = c.awards
    cand_sentence = cand.context.sentence
    cand_cell = cand_sentence.cell
    if len(cand_sentence) >= 2:
        return TRUE
    else:
        return FALSE

@labeling_function()
def LF_awards_dummy(c):
    return TRUE

In [ ]:
president_name_awards_lfs = [
    LF_awards_has_link,
    LF_awards_is_full_sentence,
    LF_awards_sentence_num,
    LF_awards_dummy
]

### ラベリング関数の適用

すべての学習用`candidate`に対して実際にLFを実行し、`Labels`と`LabelKeys`のセット(LFの名前のみ)をデータベースに生成する必要があります。
このとき、既存の`Labels`と`LabelKeys`はすべて削除されることに注意してください。

[ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/supervision.html#fonduer.supervision.Labeler)で`Labeler`が提供するAPIを見ることができます。

In [ ]:
%time labeler_pob_city.apply(split=0, lfs=[president_name_pob_city_lfs], train=True, parallelism=PARALLEL)
%time L_train_pob_city = labeler_pob_city.get_label_matrices(train_cands_Presidentname_Placeofbirth)

In [ ]:
%time labeler_pob_state.apply(split=0, lfs=[president_name_pob_state_lfs], train=True, parallelism=PARALLEL)
%time L_train_pob_state = labeler_pob_state.get_label_matrices(train_cands_Presidentname_Placeofbirth)

In [ ]:
%time labeler_predecessor.apply(split=0, lfs=[president_name_predecessor_lfs], train=True, parallelism=PARALLEL)
%time L_train_predecessor = labeler_predecessor.get_label_matrices(train_cands_Presidentname_Predecessor)

In [ ]:
%time labeler_successor.apply(split=0, lfs=[president_name_successor_lfs], train=True, parallelism=PARALLEL)
%time L_train_successor = labeler_successor.get_label_matrices(train_cands_Presidentname_Successor)

In [ ]:
%time labeler_awards.apply(split=0, lfs=[president_name_awards_lfs], train=True, parallelism=PARALLEL)
%time L_train_awards = labeler_awards.get_label_matrices(train_cands_Presidentname_Awards)

得られたラベルマトリックスに関する統計も見ることができます。
`Coverage`は、ラベリング関数が非ゼロのラベルを発行した`candidate`の割合です。
`Overlap`は、ラベリング関数が0以外のラベルを発行した`candidate`と、他のラベリング関数が0以外のラベルを発行した`candidate`の割合です。
`Conflict`は、ラベリング関数が非ゼロのラベルを発行し、別のラベリング関数が非ゼロのラベルと衝突する`candidate`の割合です。

また、すでにゴールドラベルを読み込んでいるので、[Snorkel](https://github.com/snorkel-team/snorkel)の`LFAnalysis`モジュールを使って、これらのラベリング関数とゴールドラベルを比較し、その経験的な精度を見ることができます。

In [ ]:
L_gold_train = labeler_pob_city.get_gold_labels(train_cands_Presidentname_Placeofbirth, annotator="gold")

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(
    L_train_pob_city[0],
    lfs=sorted(president_name_pob_city_lfs, key=lambda lf: lf.name)
).lf_summary(Y=L_gold_train[0].reshape(-1))

### 生成モデルの適用

次に、LFのモデルを学習し、その精度を推定します。
モデルの学習が完了したら、LFの出力を組み合わせて、ノイズを考慮した単一の学習ラベルセットを作成し、抽出器に使用することができます。
直感的には、LFがどのように重なり合い、どのように衝突するかを観察することで、LFをモデル化することになります。
そのために、[Snorkel](https://github.com/snorkel-team/snorkel)のシングルタスクラベルモデルを利用します。

In [ ]:
from snorkel.labeling.model import LabelModel

gen_model = LabelModel(cardinality=2)
%time gen_model.fit(L_train_pob_city[0], n_epochs=500, log_freq=100)

ここで、学習`candidate`に生成モデルを適用し、ノイズを考慮した学習ラベルセットを取得します。
これを学習用マージナルと呼ぶことにします。

In [ ]:
train_marginals = gen_model.predict_proba(L_train_pob_city[0])

トレーニングマージンの分布を見てみます。

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_marginals[:, TRUE], bins=20)
plt.show()

### ラベリング関数を反復するためのモデル使用

生成モデルの学習が終わったので、ここで一旦停止し、ラベリング関数セットのデバッグや改良を行うことができます。
まず、LFを開発セットに適用していきます。

In [ ]:
labeler_pob_city.apply(split=1, lfs=[president_name_pob_city_lfs], parallelism=PARALLEL)
%time L_dev_pob_city = labeler_pob_city.get_label_matrices(dev_cands_Presidentname_Placeofbirth)

In [ ]:
L_dev_pob_city[0].shape

### 生成モデルの性能の解釈

この時点で、開発セットのF1スコアは約0.6～0.7となり、かなり良い値になっています。
しかし、この解釈には十分な注意が必要です。
この開発セットを参考にラベリング関数を開発し、そのラベリング関数で生成モデルを構成しているので、ここでは非常に良いスコアが期待できるのです。

実際には、この集合に対して多少過剰に適合していると思われます。
しかし、これは問題ありません。
次回は、開発セットを超えて汎化する、より強力な末端抽出モデルを学習し、ブラインドテストセット(開発中に一度も見ていないセット)で評価していきます。

### 識別モデルのトレーニング

最後に生成したノイズの多い学習ラベルを使って、抽出モデルを学習します。
このチュートリアルでは、シンプルでありながら、かなり効果的なロジスティック回帰モデルを学習します。

各`candidate`をtrueまたはfalseのメンションとして分類する識別モデルを訓練するために、訓練マージンを使用します。

Fonduerでは、新しい機械学習フレームワーク[Emmental](https://github.com/SenWu/emmental)を使用して、すべてのモデル学習をサポートします。

In [ ]:
import emmental

# Setup training config
config = {
    "meta_config": {"verbose": True},
    "model_config": {"model_path": None, "device": 0, "dataparallel": False},
    "learner_config": {
        "n_epochs": 30,
        "optimizer_config": {"lr": 0.001, "l2": 0.0},
        "task_scheduler": "round_robin",
    },
    "logging_config": {
        "evaluation_freq": 1,
        "counter_unit": "epoch",
        "checkpointing": False,
        "checkpointer_config": {
            "checkpoint_metric": {f"{ATTRIBUTE}/{ATTRIBUTE}/train/loss": "min"},
            "checkpoint_freq": 1,
            "checkpoint_runway": 2,
            "clear_intermediate_checkpoints": True,
            "clear_all_checkpoints": True,
        },
    },
}

emmental.init(Meta.log_path)
emmental.Meta.update_config(config=config)

In [ ]:
# Collect word counter from training data
from fonduer.learning.utils import collect_word_counter

word_counter = collect_word_counter(train_cands_Presidentname_Placeofbirth)

In [ ]:
# Generate word embedding module for LSTM model
# (in Logistic Regression, we generate it since Fonduer dataset requires word2id dict)
from emmental.modules.embedding_module import EmbeddingModule

arity = 2

# Geneate special tokens
specials = []
for i in range(arity):
    specials += [f"~~[[{i}", f"{i}]]~~"]

emb_layer = EmbeddingModule(
    word_counter=word_counter, word_dim=300, specials=specials
)

In [ ]:
# Generate dataloader for training set
from emmental.data import EmmentalDataLoader
from fonduer.learning.dataset import FonduerDataset
import numpy as np

# Filter out noise samples
diffs = train_marginals.max(axis=1) - train_marginals.min(axis=1)
train_idxs = np.where(diffs > 1e-6)[0]

train_dataloader = EmmentalDataLoader(
    task_to_label_dict={ATTRIBUTE: "labels"},
    dataset=FonduerDataset(
        ATTRIBUTE,
        train_cands_Presidentname_Placeofbirth[0],
        F_train_Presidentname_Placeofbirth[0],
        emb_layer.word2id,
        train_marginals,
        train_idxs,
    ),
    split="train",
    batch_size=100,
    shuffle=True,
)

In [ ]:
# Creaet task and model, and perform training
from emmental.model import EmmentalModel
from fonduer.learning.task import create_task
from emmental.learner import EmmentalLearner

tasks = create_task(
    ATTRIBUTE, 2, F_train_Presidentname_Placeofbirth[0].shape[1], 2, emb_layer, model="LogisticRegression"
)

model = EmmentalModel(name=f"{ATTRIBUTE}_task")

for task in tasks:
    model.add_task(task)

emmental_learner = EmmentalLearner()
emmental_learner.learn(model, [train_dataloader])

## テストセットでの評価
この最後のセクションでは、ブラインドテストセットにおける抽出モデルのパフォーマンスを得ます(split 2)。
まず、先ほどのテストセットのラベルとゴールド`candidate`を読み込みます。

In [ ]:
import numpy as np
from wiki_table_utils import entity_level_f1

今度は、判別モデルを使ったスコアリングを行います。

In [ ]:
# Generate dataloader for test data
test_dataloader = EmmentalDataLoader(
    task_to_label_dict={ATTRIBUTE: "labels"},
    dataset=FonduerDataset(
        ATTRIBUTE, test_cands_Presidentname_Placeofbirth[0], F_test_Presidentname_Placeofbirth[0], emb_layer.word2id, 2
    ),
    split="test",
    batch_size=100,
    shuffle=False,
)

In [ ]:
gold_file = "../../tutorial_president_wiki/data/president_tutorial_gold.csv"

test_preds = model.predict(test_dataloader, return_preds=True)
positive = np.where(np.array(test_preds["probs"][ATTRIBUTE])[:, TRUE] > 0.6)
true_pred = [test_cands_Presidentname_Placeofbirth[0][_] for _ in positive[0]]
%time (TP, FP, FN) = entity_level_f1(true_pred, gold_file, test_docs)